In [2]:
import numpy as np
import pandas as pd
from IPython.display import display

# 13 The Black-Litterman Approach to Portfolio Optimization

### 13.2 A Naive Problem

In [3]:
SuperDuper = pd.read_excel("Chapter 13.xlsm", header=None)
SuperDuper.shape

(67, 12)

In [4]:
info = pd.DataFrame(SuperDuper.loc[2:3,1:10].values, columns=SuperDuper.loc[1,1:10].values, index=["volume","ratio"])

In [5]:
info

,General Motors GM,Home Depot HD,International Paper IP,Hewlett-Packard HPQ,Altria MO,American Express AXP,Alcoa Aluminum AA,DuPont DD,Merck MRK,MMM
volume,16.85,73.98,15.92,88.37,153.33,65.66,28.16,38.32,79.51,60.9
ratio,0.0271337,0.11913,0.0256361,0.142303,0.246908,0.105733,0.0453462,0.0617069,0.128035,0.0980676


In [7]:
data_raw = pd.DataFrame(SuperDuper.loc[6:,1:10].values, index=SuperDuper.loc[6:,0].values, columns=SuperDuper.loc[1,1:10].values)

In [8]:
data_raw.head()

,General Motors GM,Home Depot HD,International Paper IP,Hewlett-Packard HPQ,Altria MO,American Express AXP,Alcoa Aluminum AA,DuPont DD,Merck MRK,MMM
2001-06-01,50.31,45.26,31.22,26.47,38.74,32.47,36.06,40.88,50.74,51.56
2001-07-02,49.72,48.26,35.64,22.83,35.61,33.82,35.37,36.29,53.98,50.55
2001-08-01,43.14,44.06,35.32,21.48,37.1,30.54,34.5,35.01,51.96,47.3
2001-09-04,33.81,36.79,30.66,14.92,37.79,24.37,28.06,32.07,53.16,44.71
2001-10-01,32.56,36.66,31.5,15.65,36.63,24.75,29.34,34.18,50.93,47.43


In [41]:
data = data_raw.pct_change().dropna()


In [42]:
info.loc["mean return"] = data.mean(axis=0)
info.loc["std"] = data.std()
info

,General Motors GM,Home Depot HD,International Paper IP,Hewlett-Packard HPQ,Altria MO,American Express AXP,Alcoa Aluminum AA,DuPont DD,Merck MRK,MMM
volume,16.85,73.98,15.92,88.37,153.33,65.66,28.16,38.32,79.51,60.9
ratio,0.0271337,0.11913,0.0256361,0.142303,0.246908,0.105733,0.0453462,0.0617069,0.128035,0.0980676
mean return,-0.00289543,-0.000385766,0.00251992,0.00885497,0.0144841,0.0103157,0.00272343,0.00216985,-0.00231611,0.00905608
std,0.109221,0.0846084,0.0634188,0.110165,0.0871669,0.063773,0.0957484,0.0619802,0.0795936,0.0564949


In [11]:
E = info.loc["mean return"].values

S = (data-E).T.dot(data) / (data.shape[0]-1)
rf = 0.004
print(S.shape)
print(E.shape)

(10, 10)
(10,)


In [12]:
E

array([-0.0028954300589064323, -0.00038576563712894676,
       0.002519920959113198, 0.008854965595154014, 0.014484135718647576,
       0.010315691303775434, 0.0027234317712828376, 0.00216985418435956,
       -0.0023161094085593597, 0.009056081319677603], dtype=object)

In [25]:
def get_opt_port(S,E,c):
    S_inv = S_inv = pd.DataFrame(np.linalg.inv(S.values), index=S.index, columns=S.columns)
    tmp = S_inv.dot(E-rf)
    optport = tmp/tmp.sum()
    return optport

In [26]:
optport = get_opt_port(S,E,rf)
optport

General Motors\nGM         -1.24884
Home Depot\nHD             -3.36966
International Paper\nIP    -2.17696
Hewlett-Packard\nHPQ      -0.249645
Altria\nMO                  1.45462
American Express\nAXP       5.65314
Alcoa Aluminum\nAA          1.11991
DuPont\nDD                 -2.28426
Merck\nMRK                 -1.37325
MMM                         3.47496
dtype: object

In [32]:
#use shrinkage
lam = 0.3
S2 = np.zeros(S.shape)
for i in range(S.shape[0]):
    for j in range(S.shape[1]):
        if i==j:
            S2[i,j] = S.values[i,j]
S2_df = pd.DataFrame(S2,index=S.index, columns=S.columns)
S_shrinkage = lam*S + (1-lam)*S2_df
#S_shrinkage

,General Motors GM,Home Depot HD,International Paper IP,Hewlett-Packard HPQ,Altria MO,American Express AXP,Alcoa Aluminum AA,DuPont DD,Merck MRK,MMM
General Motors\nGM,0.011929,0.000852,0.000719,0.001216,0.000327,0.000924,0.001367,0.000553,0.000352,0.000400
Home Depot\nHD,0.000852,0.007159,0.000533,0.001286,0.000648,0.000965,0.001324,0.000595,0.000062,0.000535
International Paper\nIP,0.000719,0.000533,0.004022,0.000935,0.000018,0.000687,0.001297,0.000636,0.000388,0.000482
Hewlett-Packard\nHPQ,0.001216,0.001286,0.000935,0.012136,0.000768,0.001430,0.001841,0.001036,0.000692,0.000717
Altria\nMO,0.000327,0.000648,0.000018,0.000768,0.007598,0.000449,0.000493,0.000251,0.000162,0.000220
American Express\nAXP,0.000924,0.000965,0.000687,0.001430,0.000449,0.004067,0.001100,0.000588,0.000342,0.000410
Alcoa Aluminum\nAA,0.001367,0.001324,0.001297,0.001841,0.000493,0.001100,0.009168,0.001226,0.000579,0.000749
DuPont\nDD,0.000553,0.000595,0.000636,0.001036,0.000251,0.000588,0.001226,0.003842,0.000542,0.000586
Merck\nMRK,0.000352,0.000062,0.000388,0.000692,0.000162,0.000342,0.000579,0.000542,0.006335,0.000161
MMM,0.000400,0.000535,0.000482,0.000717,0.000220,0.000410,0.000749,0.000586,0.000161,0.003192


In [34]:
optport = get_opt_port(S_shrinkage,E,rf)
optport

General Motors\nGM        -0.545983
Home Depot\nHD            -0.781378
International Paper\nIP   -0.371663
Hewlett-Packard\nHPQ       0.285207
Altria\nMO                  1.10737
American Express\nAXP       1.46742
Alcoa Aluminum\nAA        -0.146076
DuPont\nDD                -0.545697
Merck\nMRK                -0.867197
MMM                         1.39799
dtype: object

### 13.3 Black and Litterman's Solution to the Optimization Problem